In [8]:
import pandas as pd
import numpy as np

# 1. Load & initial cleanup
df = pd.read_csv("sample_data.csv")
df.columns = df.columns.str.strip()
df['END DATE'] = pd.to_datetime(df['END DATE'], errors='coerce')

# 2. Ensure numeric prices & create SKU key
price_cols = ['NON-PROMO PRICE', 'PROMO PRICE', 'AVG PRICE']
df[price_cols] = df[price_cols].apply(pd.to_numeric, errors='coerce')

df['Brand_Pack_PackType'] = (
    df['BRAND'] + ' x ' +
    df['ACTUAL PACK SIZE'].astype(str) + ' x ' +
    df['PACK TYPE'].astype(str)
)

# 3. Baseline price (80th percentile non-promo price)
baseline_price = (
    df.groupby(['Markets','YEAR','Brand_Pack_PackType'])['NON-PROMO PRICE']
      .quantile(0.8)
      .reset_index(name='Baseline Price')
)
df = df.merge(baseline_price, on=['Markets','YEAR','Brand_Pack_PackType'], how='left')

# 4. Identify promo weeks
df['Promo Vol Check'] = (df['Sales Units Any Promo'] > 0.5 * df['Sales Units']).map({True:'Y', False:'N'})
df['Promo 5% Deviation Check'] = (df['PROMO PRICE'] < 0.95 * df['NON-PROMO PRICE']).map({True:'Y', False:'N'})
df['Promo Week Check'] = ((df['Promo Vol Check']=='Y') & (df['Promo 5% Deviation Check']=='Y')).map({True:'Y', False:'N'})

# 5. Baseline volume (mean non-promo units)
df['Sales Units No Promo'] = pd.to_numeric(df['Sales Units No Promo'], errors='coerce')
non_promo = df[df['Promo Week Check']=='N']
baseline_vol = (
    non_promo.groupby(['Markets','YEAR','Brand_Pack_PackType'])['Sales Units No Promo']
             .mean()
             .reset_index(name='Baseline Volume')
)
df = df.merge(baseline_vol, on=['Markets','YEAR','Brand_Pack_PackType'], how='left')

# 6. Filter anomalies & compute uplift
df['Erase Anomaly'] = ((df['Baseline Volume']>df['Sales Units']) & (df['Baseline Price']<df['PROMO PRICE'])).map({True:'Y', False:'N'})
df['Uplift'] = df.apply(
    lambda r: r['Sales Units'] - r['Baseline Volume']
              if (r['Promo Week Check']=='Y') and (r['Erase Anomaly']=='N')
              else np.nan,
    axis=1
)

# 7. Create a stable “promo depth” column on df before slicing
df['promo_depth_pct'] = 1.0 - (df['PROMO PRICE'] / df['Baseline Price'])

# 8. Build valid promo-week table
valid = df[(df['Promo Week Check']=='Y') & (df['Erase Anomaly']=='N')].copy()
valid.sort_values(['Markets','YEAR','Brand_Pack_PackType','END DATE'], inplace=True)
valid['Date_Gap'] = valid.groupby(['Markets','YEAR','Brand_Pack_PackType'])['END DATE'].diff().dt.days.fillna(0)
valid['New_Event'] = (valid['Date_Gap']>7).astype(int)
valid['Event_ID'] = valid.groupby(['Markets','YEAR','Brand_Pack_PackType'])['New_Event'].cumsum()
valid['Valid Event Uplift'] = ((valid['Uplift'].notna()) & (valid['Uplift']>0.2*valid['Baseline Volume'])).map({True:'Y',False:'N'})
valid = valid[valid['Valid Event Uplift']=='Y']

# 9. Compute event-level median depth and bucket
event_depth = (
    valid
      .groupby(['Markets','YEAR','Brand_Pack_PackType','Event_ID'])['promo_depth_pct']
      .median()
      .reset_index(name='event_median_depth')
)

bins = [0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,
        0.50,0.55,0.60,0.65,0.70,0.75,0.80,1.10]
labels = ['5–9.99%','10–14.99%','15–19.99%','20–24.99%','25–29.99%','30–34.99%',
          '35–39.99%','40–44.99%','45–49.99%','50–54.99%','55–59.99%','60–64.99%',
          '65–69.99%','70–74.99%','75–79.99%','80%+']
event_depth['depth_bucket'] = pd.cut(
    event_depth['event_median_depth'],
    bins=bins,
    labels=labels,
    right=False
)

valid = valid.merge(
    event_depth,
    on=['Markets','YEAR','Brand_Pack_PackType','Event_ID'],
    how='left'
)

# 10. Compute event duration & duration buckets
event_duration = valid.groupby(['Markets','YEAR','Brand_Pack_PackType','Event_ID']) \
                      .size() \
                      .reset_index(name='Event Duration (weeks)')
valid = valid.merge(
    event_duration,
    on=['Markets','YEAR','Brand_Pack_PackType','Event_ID'],
    how='left'
)

event_stats = (
    valid.groupby(['Markets','YEAR','Brand_Pack_PackType','Event_ID'])
         .agg(
             Event_Duration=('Event_ID','size'),
             Max_Depth=('promo_depth_pct','max')
         )
         .reset_index()
)
event_stats['Duration Bucket'] = pd.cut(
    event_stats['Event_Duration'],
    bins=[0,3,6,np.inf],
    labels=['Short','Medium','Long'],
    right=True
)
event_stats['Depth Bucket'] = pd.cut(
    event_stats['Max_Depth'],
    bins=[-1,0.10,0.20,1.10],
    labels=['Shallow','Medium','Deep'],
    right=False
)

valid = valid.merge(
    event_stats[['Markets','YEAR','Brand_Pack_PackType','Event_ID','Duration Bucket','Depth Bucket']],
    on=['Markets','YEAR','Brand_Pack_PackType','Event_ID'],
    how='left'
)

# 11. Save outputs
df.to_excel("sample_data_cleaned.xlsx", index=False)
valid.to_excel("sample_data_events.xlsx", sheet_name="Weekly", index=False)
